# **FOSSASIA Summit 2021**
# NEW (MariaDB) SQL
## Practical Examples of Modern SQL to solve harder problems

Code Examples:

* [CTEs](https://dbfiddle.uk/?rdbms=mariadb_10.3&fiddle=3ca4c4f811ac12f2d0e8b3b6c11799a8)
* [Recursive CTEs](https://dbfiddle.uk/?rdbms=mariadb_10.3&fiddle=939a43cfcd7396b1ad00c7d8cda867dc)
* [Recursive CTEs CYCLES RESTRICT](https://dbfiddle.uk/?rdbms=mariadb_10.5&fiddle=84813c2f2eb4f76938c50fe9f58ea586)
* [INSERT RETURNING](https://dbfiddle.uk/?rdbms=mariadb_10.5&fiddle=d44138556fbe1872e06fe6cb36dbeda8)
* [System Versioned Tables - time](https://dbfiddle.uk/?rdbms=mariadb_10.3&fiddle=42260ada361656a76b59d4999f559247)
* [System Versioned Tables - transaction](https://dbfiddle.uk/?rdbms=mariadb_10.3&fiddle=35435ec490bcbd351a6dfd91e8964f57)
* [System Versioned Tables - partitioning](https://dbfiddle.uk/?rdbms=mariadb_10.5&fiddle=e6658a5663747ebbc71113a2e0f7103c)
* [IPv6 - INET6 Datatype](https://dbfiddle.uk/?rdbms=mariadb_10.5&fiddle=6cb350bfa716d577e1c1af03a54a2ee5)
* [Application Time Periods](https://dbfiddle.uk/?rdbms=mariadb_10.4&fiddle=2625f132da5536e748be6b72211c4afd)
* [Application Time Periods - without Overlaps](https://dbfiddle.uk/?rdbms=mariadb_10.5&fiddle=e7cf6718bec732f3e01505ea25e8cd7d)

## *Common Table Expressions - CTEs - MariaDB 10.2+*

In [1]:
select version();

version() 10.5.10-MariaDB

In [2]:
create database if not exists ctetests;

In [3]:
use ctetests;

In [4]:
create or replace table student_tests (
 name CHAR(10), test CHAR(10), 
 score TINYINT, test_date DATE
);

### **Adding some data into the table**

In [5]:
INSERT INTO student_tests 
 (name, test, score, test_date) VALUES
 ('Chun', 'SQL', 75, '2012-11-05'), 
 ('Chun', 'Tuning', 73, '2013-06-14'),
 ('Esben', 'SQL', 43, '2014-02-11'), 
 ('Esben', 'Tuning', 31, '2014-02-09'), 
 ('Kaolin', 'SQL', 56, '2014-01-01'),
 ('Kaolin', 'Tuning', 88, '2013-12-29'), 
 ('Tatiana', 'SQL', 75, '2012-04-28'), 
 ('Tatiana', 'Tuning', 83, '2013-09-30');

### **Running a CTE**

In [6]:
WITH sqltest AS 
   ( SELECT *
     FROM student_tests
     WHERE test = 'SQL'
)
SELECT *
FROM sqltest;

name test score test_date Chun SQL 75 2012-11-05 Esben SQL 43 2014-02-11 Kaolin SQL 56 2014-01-01 Tatiana SQL 75 2012-04-28

In [7]:
WITH sqltest AS 
   ( SELECT *
     FROM student_tests
     WHERE test = 'SQL'
)
SELECT *
FROM sqltest t1
JOIN sqltest t2 ON t1.score=t2.score
WHERE  t1.name != t2.name
AND t1.name < t2.name;

name test score test_date name test score test_date Chun SQL 75 2012-11-05 Tatiana SQL 75 2012-04-28

# **Recursive CTEs** (MariaDB-10.2+)

In [8]:
use ctetests;

In [9]:
CREATE OR REPLACE TABLE bus_routes (origin varchar(50), dst varchar(50));

In [17]:
INSERT INTO bus_routes VALUES 
  ('New York', 'Boston'), 
  ('Boston', 'New York'), 
  ('New York', 'Washington'), 
  ('Washington', 'Boston'), 
  ('Washington', 'Raleigh'),
  ('Boston', 'Sydney');

In [11]:
set max_recursive_iterations = 20;

## Form of Recursive CTEs 



In [12]:
WITH RECURSIVE cte AS (
   SELECT anchor_data    -- This deterimines types, field names, and initial data of `cte`
   FROM ....
   UNION [ALL]
   SELECT recursive_part
   FROM .....
)
SELECT ... FROM cte;

### Where can I go from New York

In [18]:
WITH RECURSIVE bus_dst AS ( 
    SELECT origin as dst
    FROM bus_routes
    WHERE origin='New York' 
  UNION
    SELECT bus_routes.dst
    FROM bus_routes
    JOIN bus_dst ON bus_dst.dst= bus_routes.origin 
) 
SELECT * FROM bus_dst;

dst New York Boston Washington Raleigh Sydney

### Watch out for casting

In [26]:
WITH RECURSIVE tbl AS (
  SELECT NULL AS col
  UNION
  SELECT "THIS NEVER SHOWS UP" AS col FROM tbl
)
SELECT col FROM tbl;

col NULL

In [28]:
WITH RECURSIVE tbl AS (
  SELECT CAST(NULL AS CHAR(50)) AS col
  UNION SELECT "THIS WILL SHOW UP" AS col FROM tbl
)  
SELECT col FROM tbl;

col NULL THIS WILL SHOW UP

# Recursive CTEs CYCLE RESTRICT (MariaDB 10.5+)

In [29]:
CREATE TABLE t1 (from_ int, to_ int);

In [31]:
INSERT INTO t1 VALUES (1,2), (1,100), (2,3), (3,4), (4,1);

In [32]:
SELECT * FROM t1;

from_ to_ 1 2 1 100 2 3 3 4 4 1

In [35]:
WITH RECURSIVE cte (depth, distance, from_, to_) AS ( 
  SELECT 0,0,1,1
  UNION DISTINCT
  SELECT depth+1, distance + abs(t1.from_ - t1.to_), t1.from_, t1.to_ 
    FROM t1
    JOIN cte ON t1.from_ = cte.to_ 
) 
SELECT * FROM cte;

depth distance from_ to_ 0 0 1 1 1 1 1 2 1 99 1 100 2 2 2 3 3 3 3 4 4 6 4 1 5 7 1 2 5 105 1 100 6 8 2 3 7 9 3 4 8 12 4 1 9 13 1 2 9 111 1 100 10 14 2 3 11 15 3 4 12 18 4 1 13 19 1 2 13 117 1 100 14 20 2 3 15 21 3 4 16 24 4 1 17 25 1 2 17 123 1 100 18 26 2 3 19 27 3 4 20 30 4 1

In [36]:
WITH RECURSIVE cte (depth, distance, from_, to_) AS ( 
  SELECT 0,0,1,1
  UNION DISTINCT
  SELECT depth+1, distance + abs(t1.from_ - t1.to_), t1.from_, t1.to_ 
    FROM t1
    JOIN cte ON t1.from_ = cte.to_ 
)
SELECT * FROM cte;

depth distance from_ to_ 0 0 1 1 1 1 1 2 1 99 1 100 2 2 2 3 3 3 3 4 4 6 4 1 5 7 1 2 5 105 1 100 6 8 2 3 7 9 3 4 8 12 4 1 9 13 1 2 9 111 1 100 10 14 2 3 11 15 3 4 12 18 4 1 13 19 1 2 13 117 1 100 14 20 2 3 15 21 3 4 16 24 4 1 17 25 1 2 17 123 1 100 18 26 2 3 19 27 3 4 20 30 4 1

In [37]:
WITH RECURSIVE cte (depth, distance, from_, to_) AS ( 
  SELECT 0,0,1,1
  UNION DISTINCT
  SELECT depth+1, distance + abs(t1.from_ - t1.to_), t1.from_, t1.to_ 
    FROM t1
    JOIN cte ON t1.from_ = cte.to_ 
)
CYCLE from_, to_ RESTRICT
SELECT * FROM cte;

depth distance from_ to_ 0 0 1 1 1 1 1 2 1 99 1 100 2 2 2 3 3 3 3 4 4 6 4 1

# Insert RETURNING (MariaDB 10.5+)
`RELACE ... RETURNING`  (MariaDB 10.5+)
`DELETE ... RETURNING`  (MariaDB 10.3+)

In [101]:
create database ins;

ERROR 1007 (HY000): Can't create database 'ins'; database exists


In [102]:
use ins;

In [103]:
create or replace table users (
id int unsigned not null auto_increment primary key,
name varchar(30));

In [104]:
insert into users (name) values ('bob'), ('jane'), ('fred'), ('harry')
returning id;

id 1 2 3 4

In [105]:
alter table users add unique key(name);

In [106]:
insert /* ignore */ into users (name)
values ('john'), ('bob'), ('sarah'), ('carmel')
returning id,name;

id name 5 john 6 sarah 7 carmel

# System Versioned Tables (MariaDB-10.3)

In [70]:
create database sv;

In [115]:
use sv;


SQL:2011 standard form

In [116]:
CREATE OR REPLACE TABLE t(
   x INT,
   start_timestamp TIMESTAMP(6) GENERATED ALWAYS AS ROW START,
   end_timestamp TIMESTAMP(6) GENERATED ALWAYS AS ROW END,
   PERIOD FOR SYSTEM_TIME(start_timestamp, end_timestamp)
) WITH SYSTEM VERSIONING;

In [123]:
CREATE OR REPLACE TABLE t (
x INT
) WITH SYSTEM VERSIONING;

In [124]:
INSERT INTO t VALUES (1),(3),(5);

In [125]:
INSERT INTO t VALUES (5),(6),(8);

In [6]:
select * from t;

x 1 3 5 5 6 8

In [7]:
SELECT x, ROW_START, ROW_END FROM t;

x ROW_START ROW_END 1 2021-03-17 13:21:30.087718 2038-01-19 14:14:07.999999 3 2021-03-17 13:21:30.087718 2038-01-19 14:14:07.999999 5 2021-03-17 13:21:30.087718 2038-01-19 14:14:07.999999 5 2021-03-17 13:21:32.073561 2038-01-19 14:14:07.999999 6 2021-03-17 13:21:32.073561 2038-01-19 14:14:07.999999 8 2021-03-17 13:21:32.073561 2038-01-19 14:14:07.999999

In [126]:
delete from t where x = 5;

In [127]:
insert into t VALUES (15),(26),(48);

In [10]:
SELECT x, ROW_START, ROW_END FROM t;

x ROW_START ROW_END 1 2021-03-17 13:21:30.087718 2038-01-19 14:14:07.999999 3 2021-03-17 13:21:30.087718 2038-01-19 14:14:07.999999 6 2021-03-17 13:21:32.073561 2038-01-19 14:14:07.999999 8 2021-03-17 13:21:32.073561 2038-01-19 14:14:07.999999 15 2021-03-17 13:21:47.445882 2038-01-19 14:14:07.999999 26 2021-03-17 13:21:47.445882 2038-01-19 14:14:07.999999 48 2021-03-17 13:21:47.445882 2038-01-19 14:14:07.999999

In [12]:
SELECT x, ROW_START, ROW_END
FROM t
FOR SYSTEM_TIME ALL;

x ROW_START ROW_END 1 2021-03-17 13:21:30.087718 2038-01-19 14:14:07.999999 3 2021-03-17 13:21:30.087718 2038-01-19 14:14:07.999999 5 2021-03-17 13:21:30.087718 2021-03-17 13:21:44.563212 5 2021-03-17 13:21:32.073561 2021-03-17 13:21:44.563212 6 2021-03-17 13:21:32.073561 2038-01-19 14:14:07.999999 8 2021-03-17 13:21:32.073561 2038-01-19 14:14:07.999999 15 2021-03-17 13:21:47.445882 2038-01-19 14:14:07.999999 26 2021-03-17 13:21:47.445882 2038-01-19 14:14:07.999999 48 2021-03-17 13:21:47.445882 2038-01-19 14:14:07.999999

In [14]:
SELECT *
FROM t
FOR SYSTEM_TIME AS OF TIMESTAMP '2021-03-17 13:21:44';

x 1 3 5 5 6 8

In [17]:
SELECT *
FROM t
FOR SYSTEM_TIME FROM '2021-03-17 13:21:45' TO '2021-03-17 13:21:47.445882';

x 1 3 6 8

In [118]:
ALTER TABLE t DROP SYSTEM VERSIONING;

In [93]:
SELECT * FROM t FOR SYSTEM_TIME AS OF TIMESTAMP '2021-03-17 12:32:33.850974';

ERROR 4124 (HY000): Table `t` is not system-versioned


In [119]:
ALTER TABLE t ADD COLUMN ts TIMESTAMP(6) GENERATED ALWAYS AS ROW START,
              ADD COLUMN te TIMESTAMP(6) GENERATED ALWAYS AS ROW END,
              ADD PERIOD FOR SYSTEM_TIME(ts, te),
              ADD SYSTEM VERSIONING;

Stage: 2 of 2 'Enabling keys' 0% of stage done

In [130]:
select x from t;

x 1 3 6 8 15 26 48

## Transaction Precise History in Innodb

In [20]:
CREATE OR REPLACE TABLE tinnodb (
   x INT,
   start_trxid BIGINT UNSIGNED GENERATED ALWAYS AS ROW START,
   end_trxid BIGINT UNSIGNED GENERATED ALWAYS AS ROW END,
   PERIOD FOR SYSTEM_TIME(start_trxid, end_trxid)
) WITH SYSTEM VERSIONING;

In [21]:
insert into tinnodb (x) values (4), (22), (33);

In [22]:
select * from tinnodb;

x start_trxid end_trxid 4 361 18446744073709551615 22 361 18446744073709551615 33 361 18446744073709551615

In [23]:
begin;
insert into tinnodb (x) values (124), (222), (133);
insert into tinnodb (x) values (44), (422), (433);
commit;

In [25]:
select * from tinnodb;

x start_trxid end_trxid 4 361 18446744073709551615 22 361 18446744073709551615 33 361 18446744073709551615 124 368 18446744073709551615 222 368 18446744073709551615 133 368 18446744073709551615 44 368 18446744073709551615 422 368 18446744073709551615 433 368 18446744073709551615

In [27]:
select *
from tinnodb 
FOR SYSTEM_TIME AS OF TRANSACTION 368;


x start_trxid end_trxid 4 361 18446744073709551615 22 361 18446744073709551615 33 361 18446744073709551615 124 368 18446744073709551615 222 368 18446744073709551615 133 368 18446744073709551615 44 368 18446744073709551615 422 368 18446744073709551615 433 368 18446744073709551615

In [41]:
CREATE OR REPLACE TABLE inv (
   x INT,
   start_trxid BIGINT UNSIGNED GENERATED ALWAYS AS ROW START INVISIBLE,
   end_trxid BIGINT UNSIGNED GENERATED ALWAYS AS ROW END INVISIBLE,
   PERIOD FOR SYSTEM_TIME(start_trxid, end_trxid)
) WITH SYSTEM VERSIONING;

In [46]:
insert into inv select x from tinnodb;

In [47]:
select * from inv;

x 4 22 33 124 222 133 44 422 433

In [48]:
select x, start_trxid, end_trxid FROM inv;

x start_trxid end_trxid 4 393 18446744073709551615 22 393 18446744073709551615 33 393 18446744073709551615 124 393 18446744073709551615 222 393 18446744073709551615 133 393 18446744073709551615 44 393 18446744073709551615 422 393 18446744073709551615 433 393 18446744073709551615

## Paritioning Storing history separate

In [51]:
alter table t  PARTITION BY SYSTEM_TIME (
    PARTITION p_hist HISTORY,
    PARTITION p_cur CURRENT
  );

Stage: 2 of 2 'Enabling keys' 0% of stage done

Time based only, not Innodb transaction-precise

Must have one `CURRENT` parition

In [ ]:
CREATE TABLE tvlimt (x INT) WITH SYSTEM VERSIONING
  PARTITION BY SYSTEM_TIME LIMIT 100000 (
    PARTITION p0 HISTORY,
    PARTITION p1 HISTORY,
    PARTITION pcur CURRENT
  );

 history rows into partition p0, and
 
 when it reaches a size of 100000 rows, MariaDB will switch to partition p1
 
 so when p1 overflows, MariaDB will issue a warning, but will continue writing into it

In [132]:
CREATE OR REPLACE TABLE tinerval (x INT) WITH SYSTEM VERSIONING 
  PARTITION BY SYSTEM_TIME 
    INTERVAL 1 MONTH 
    PARTITIONS 12;

# INET6 datatype (MariaDB-10.5)

In [55]:
create database inet6;

In [57]:
use inet6;

In [58]:
CREATE TABLE t1 (a INET6);

In [59]:
INSERT INTO t1 VALUES ('2001:db8::ff00:42:8329');

In [60]:
INSERT INTO t1 VALUES ('2001:0db8:0000:0000:0000:ff00:0042:8329');

In [62]:
INSERT INTO t1 VALUES ('::ffff:192.0.2.128');
INSERT INTO t1 VALUES ('::192.0.2.128');

In [63]:
select * from t1;

a 2001:db8::ff00:42:8329 2001:db8::ff00:42:8329 ::ffff:192.0.2.128 ::192.0.2.128

In [64]:
SELECT CAST(a AS DECIMAL) FROM t1;

ERROR 4079 (HY000): Illegal parameter data type inet6 for operation 'decimal_typecast'


In [70]:
create table t2 (a varbinary(16));

In [71]:
insert into t2 select * from t1;

In [73]:
select hex(a) from t2;

hex(a) 20010DB8000000000000FF0000428329 20010DB8000000000000FF0000428329 00000000000000000000FFFFC0000280 000000000000000000000000C0000280

In [77]:
select hex(t2.a), t1.a
from t1
join t2 using (a);

hex(t2.a) a 20010DB8000000000000FF0000428329 2001:db8::ff00:42:8329 20010DB8000000000000FF0000428329 2001:db8::ff00:42:8329 20010DB8000000000000FF0000428329 2001:db8::ff00:42:8329 20010DB8000000000000FF0000428329 2001:db8::ff00:42:8329 00000000000000000000FFFFC0000280 ::ffff:192.0.2.128 000000000000000000000000C0000280 ::192.0.2.128

In [80]:
 CREATE OR REPLACE TABLE tg (a INET6, b VARCHAR(64));

In [82]:
INSERT INTO tg VALUES (NULL,'2001:db8::ff00:42:8328');
INSERT INTO tg VALUES (NULL,'2001:db8::ff00:42:832a garbage');

In [83]:
  SELECT COALESCE(a,b) FROM tg;

COALESCE(a,b) 2001:db8::ff00:42:8328 NULL

In [84]:
show warnings;

Level Code Message Warning 1292 Incorrect inet6 value: '2001:db8::ff00:42:832a garbage'

# Application Time Periods (MariaDB 10.4+)

In [87]:
create database apptime;

In [88]:
use apptime;

In [89]:
CREATE TABLE t1(
   name VARCHAR(50), 
   date_1 DATE,
   date_2 DATE,
   PERIOD FOR date_period(date_1, date_2));

In [90]:
INSERT INTO t1 (name, date_1, date_2) VALUES
    ('a', '1999-01-01', '2000-01-01'),
    ('b', '1999-01-01', '2018-12-12'),
    ('c', '1999-01-01', '2017-01-01'),
    ('d', '2017-01-01', '2019-01-01');

In [91]:
select * from t1;

name date_1 date_2 a 1999-01-01 2000-01-01 b 1999-01-01 2018-12-12 c 1999-01-01 2017-01-01 d 2017-01-01 2019-01-01

In [92]:
DELETE FROM t1
FOR PORTION OF date_period
    FROM '2001-01-01' TO '2018-01-01';

In [93]:
select * from t1;

name date_1 date_2 a 1999-01-01 2000-01-01 b 1999-01-01 2001-01-01 c 1999-01-01 2001-01-01 d 2018-01-01 2019-01-01 b 2018-01-01 2018-12-12

* a is unchanged, as the range falls entirely out of the specified portion to be deleted.
* b, with values ranging from 1999 to 2018, is split into two rows, 1999 to 2000 and 2018-01 to 2018-12.
* c, with values ranging from 1999 to 2017, where only the upper value falls within the portion to be deleted, has been shrunk to 1999 to 2001.
* d, with values ranging from 2017 to 2019, where only the lower value falls within the portion to be deleted, has been shrunk to 2018 to 2019. 

#### constraints

* The FROM...TO clause must be constant
* Multi-delete is not supported 

In [95]:
TRUNCATE t1;

INSERT INTO t1 (name, date_1, date_2) VALUES
    ('a', '1999-01-01', '2000-01-01'),
    ('b', '1999-01-01', '2018-12-12'),
    ('c', '1999-01-01', '2017-01-01'),
    ('d', '2017-01-01', '2019-01-01');

SELECT * FROM t1;

name date_1 date_2 a 1999-01-01 2000-01-01 b 1999-01-01 2018-12-12 c 1999-01-01 2017-01-01 d 2017-01-01 2019-01-01

In [97]:
UPDATE t1 FOR PORTION OF date_period
  FROM '2000-01-01' TO '2018-01-01' 
SET name = CONCAT(name,'_original');

In [98]:

SELECT * FROM t1 ORDER BY name;

name date_1 date_2 a 1999-01-01 2000-01-01 b 1999-01-01 2000-01-01 b 2018-01-01 2018-12-12 b_original 2000-01-01 2018-01-01 c 1999-01-01 2000-01-01 c_original 2000-01-01 2017-01-01 d 2018-01-01 2019-01-01 d_original 2017-01-01 2018-01-01

# Application Time Periods - WITHOUT OVERLAPS (MariaDB 10.5+)

In [99]:
CREATE OR REPLACE TABLE rooms (
 room_number INT,
 guest_name VARCHAR(255),
 checkin DATE,
 checkout DATE,
 PERIOD FOR p(checkin,checkout)
 );
 
 INSERT INTO rooms VALUES 
 (1, 'Regina', '2020-10-01', '2020-10-03'),
 (2, 'Cochise', '2020-10-02', '2020-10-05'),
 (1, 'Nowell', '2020-10-03', '2020-10-07'),
 (2, 'Eusebius', '2020-10-04', '2020-10-06');

In [100]:
CREATE OR REPLACE TABLE rooms (
 room_number INT,
 guest_name VARCHAR(255),
 checkin DATE,
 checkout DATE,
 PERIOD FOR p(checkin,checkout),
 UNIQUE (room_number, p WITHOUT OVERLAPS)
 );
 
 INSERT INTO rooms VALUES 
 (1, 'Regina', '2020-10-01', '2020-10-03'),
 (2, 'Cochise', '2020-10-02', '2020-10-05'),
 (1, 'Nowell', '2020-10-03', '2020-10-07'),
 (2, 'Eusebius', '2020-10-04', '2020-10-06');

ERROR 1062 (23000): Duplicate entry '2-2020-10-06-2020-10-04' for key 'room_number'
